In [ ]:
! python Chrome-dino.py

In [ ]:
import numpy as np
from PIL import ImageGrab
import cv2
import time

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
game_url = "game/dino.html"
chrome_driver_path = "../chromedriver.exe"
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(200, 300)
        self._driver.get(os.path.abspath(game_url))
        if custom_config:
            self._driver.execute_script("Runner.config.ACCELERATION=0")
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        return self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def press_down(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_DOWN)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        return int(score)
    def end(self):
        self._driver.close()

In [ ]:
class DinoAgent:
    def __init__(self,game):
        self._game = game;
        self.jump();
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

In [ ]:
#processing image as required
def process_img(image):
    #image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)qqq
    #game is already in grey scale canvas, canny to get only edges and reduce unwanted objects(clouds)
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200)
    #image = resized_image = cv2.resize(image, (80, 80)) 
    image = cv2.resize(image, (0,0), fx = 0.50, fy = 0.50)
    #image = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)
    return  image

In [ ]:
def grab_screen():
    screen =  np.array(ImageGrab.grab(bbox=(0,180,400,400)))
    image = process_img(screen)
    return image

In [ ]:
print(grab_screen().shape)
# game = Game()
# dino = DinoAgent(game)
# last_time = time.time()
# while(True):
    
# #     print('loop took {} seconds'.format(time.time()-last_time))
# #     last_time = time.time()
# #     cv2.imwrite("./img_data/dino"+str(time())+".jpg",image)
# #     dino.duck()
#     #exit on q pres
#     image,r_t,end_t = get_state(game,dino,2)
# #     print('{0} {1} '.format(r_t,end_t))
#     #cv2.imshow('window',image)
#     if(dino.is_crashed()):
#         #jumping starts the game again if dino has crashed
#         print(game.get_score())
#         game.restart()
        
#     if (cv2.waitKey(25) & 0xFF == ord('q')):
#         cv2.destroyAllWindows()
#         game.end()
#         cv2.imwrite('dino.jpg',image)
#         break

In [ ]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._showImg = show_img()
        self._showImg.__next__()
            
    def get_state(self,actions=0):
        
        reward = 0.1
        is_over = False
        if actions == 1:
            self._agent.jump()
            reward = 0.1
#         elif (actions[2] == 1):
# #             self._agent.duck()
#             self._agent.jump()
#             reward = 0.1
#         if self._agent.is_crashed():
#             self._game.restart()
#             reward = -1
#             is_over = True
        image = grab_screen()
        self._showImg.send(image)
        return image

In [ ]:
def show_img():
    """
    Coroutine to store images in the "images" directory
    """
    frame = 0
    while True:
        screen = (yield)
        cv2.imshow("preview", screen)
        if (cv2.waitKey(25) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

        frame += 1

In [ ]:
def episode():
    """ 
    Coroutine of episode. 
    
    Action has to be explicitly send to this coroutine.
    """
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)
    while True:
        end = int(dino.is_crashed())
        if end :
            end *= -1
            game.restart()
            
        actions = yield game_state.get_state(), end  
        game_state.get_state(actions)



In [ ]:
import os
from random import sample as rsample

import numpy as np

from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Dense, Flatten
from keras.optimizers import SGD, RMSprop
import random
from matplotlib import pyplot as plt

In [ ]:
def experience_replay(batch_size):
    """
    Coroutine function for implementing experience replay.    
        Provides a new experience by calling "send", which in turn yields 
        a random batch of previous replay experiences.
    """
    memory = []
    while True:
        # experience is a tuple containing (S, action, reward, S_prime)
        experience = yield rsample(memory, batch_size) if batch_size <= len(memory) else None
        memory.append(experience)

In [ ]:
nb_epochs = 1000
batch_size = 128
epsilon = .8
gamma = .8
ACTIONS = 2
# Recipe of deep reinforcement learning model
model = Sequential()
model.add(Convolution2D(32,(3,3), input_shape=(110, 200,1), activation='relu'))
model.add(Convolution2D(64,(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(ACTIONS))
model.compile(RMSprop(), 'MSE')

In [ ]:
exp_replay = experience_replay(batch_size)
exp_replay.__next__()  # Start experience-replay coroutine

In [ ]:

def play_game():
    for i in range(nb_epochs):
        ep = episode()
        S, won = ep.__next__()  # Start coroutine of single entire episode
        loss = 0.
        try:
            while True:
#                 action = np.zeros(ACTIONS)
#                 action_index = random.randrange(ACTIONS) 
#                 action[action_index] = 1
                action = 0
                if np.random.random() > epsilon:
                    # Get the index of the maximum q-value of the model.
                    # Subtract one because actions are either -1, 0, or 1
                    q = model.predict(S.reshape(1,110,200,1))       
                    max_Q = np.argmax(q, axis=-1)
                    action_index = max_Q-1
#                     action[action_index] = 1
                    action = action_index
                S_prime, won = ep.send(action)
                experience = (S, action, won, S_prime)
                S = S_prime

                batch = exp_replay.send(experience)
                if batch:
                    inputs = []
                    targets = []
                    for s, a, r, s_prime in batch:
                        # The targets of unchosen actions are the q-values of the model,
                        # so that the corresponding errors are 0. The targets of chosen actions
                        # are either the rewards, in case a terminal state has been reached, 
                        # or future discounted q-values, in case episodes are still running.
                        t = model.predict(S.reshape(1,110,200,1)).flatten()
                        t[a + 1] = r
                        if not r:
                            t[a + 1] = r + gamma * model.predict(s_prime.reshape(1,110,200,1)).max(axis=-1)
                        targets.append(t)
                        inputs.append(s)
                        print('training')
                    loss += model.train_on_batch(np.array(inputs).reshape(1,110,200,1), np.array(targets))[0]

        except StopIteration:
            pass

        if (i + 1) % 100 == 0:
            print ('Epoch %i, loss: %.6f' % (i + 1, loss))


In [ ]:
play_game()